# Dipole potential 

Implempentation of a grid of 36 rod-like particles fixed in place with a dipole under an external constant isotropic eletric field. The direction of the dipole of each particle inverts its direction following a Markovian Switch given by a continuous-time Markvo state model (MSM).

In [ ]:
import os
import numpy as np
import msmrd2
import msmrd2.visualization as msmrdvis
from msmrd2.markovModels import continuousTimeMarkovStateModel as ctmsm
from msmrd2.potentials import dipole
from msmrd2.integrators import overdampedLangevinMarkovSwitch as odLangevinMS

In [ ]:
# Define continuous-time MSM
MSMtype = 0
ratematrix = np.array([[-3.0,3.0],[1.5,-1.5]])
seed = 0 # Seed = -1 used random device as seed
markovModel = ctmsm(MSMtype, ratematrix, seed)
Dlist = np.array([0.0, 0.0])
Drotlist = np.array([1.0, 1.0])
markovModel.setD(Dlist)
markovModel.setDrot(Drotlist)

In [ ]:
# Particle list definition (creates 6x6 grid of particles)
orientation = np.array([1,0,0,0])
particlelist = 36*[None]
# Create a grid of particles
for i in range(6):
    for j in range(6): 
        position = np.array([i-2.5,j-2.5,0])
        state = np.random.randint(0,2)
        part = msmrd2.particle(MSMtype, state, Dlist[state], Drotlist[state], position, orientation)   
        orientVector = np.random.normal(0,1,3)
        orientVector = orientVector/np.linalg.norm(orientVector)
        part.setOrientVector(orientVector)
        particlelist[6*i + j] = part
# Define list object that can be read by pybound functions
partlist = msmrd2.integrators.particleList(particlelist)

In [ ]:
# Over-damped Langevin integrator with Markovian Switch definition
dt = 0.001
seed = -1 # seed = -1 uses random device as seed
bodytype = "rod" # orientation has only two-degrees of freedom (vector on unit-sphere)
integrator = odLangevinMS(markovModel, dt, seed, bodytype)

In [ ]:
# Define and set potential
scalefactor = 30.0
Efieldvector = np.array([0,1,0])
potentialDipole = dipole(scalefactor, Efieldvector)
integrator.setExternalPotential(potentialDipole)

In [ ]:
# Integrate the particles, save to .xyz to produce VMD output (additional overhead)
datafile  = open('../../data/vmd/dipole.xyz', 'w')
timeIters = 20000
prevstate = [None] * len(partlist)
for i in range(timeIters):
    #datafile  = open('odLangevinDipole.xyz', 'w')
    datafile.write(str(2*len(partlist)) + '\n')
    datafile.write(str(0) + '\n')
    for j, part in enumerate(partlist):
        if part.state != prevstate[j] and prevstate[j] != None:
            part.setOrientVector(-part.orientvector)
        if part.state == 0:
            v0 = part.position
            v1 = v0 + 0.35*part.orientvector
            v2 = v0 - 0.35*part.orientvector
            datafile.write('type_0' + ' ' + ' '.join(map(str, v1)) + '\n')
            datafile.write('type_0' + ' ' + ' '.join(map(str, v2)) + '\n')
        elif part.state == 1:
            v0 = part.position
            v1 = v0 + 0.35*part.orientvector
            v2 = v0 - 0.35*part.orientvector
            datafile.write('type_1' + ' ' + ' '.join(map(str, v1)) + '\n')
            datafile.write('type_1' + ' ' + ' '.join(map(str, v2)) + '\n')
        prevstate[j] = 1*part.state
    integrator.integrate(partlist)
    if i%1000 == 0:
        print("Percentage complete: ", 100*i/timeIters, "%", end="\r")
datafile.close()
print("Percentage complete: ", 100, " %")

In [ ]:
# Generates tcl for VMD visualization all frames needs -1 (Note vmd assigns the color depending on the initial
# state. However it cannot change colors during the same session, so particles wont change color when
# changing state.
msmrdvis.generateTCL_dipole(-1, outfname = "dipole", tclfname = "../../data/vmd/dipole_2vmd.tcl")

To load the movie go to /data/vmd and run in a terminal "vmd -e potentialDipole_2vmd.tcl".

## In case VMD is not desired

In case VMD output is not desired, below we simply integrate the model and show the output directly.

In [ ]:
# Integrate the particle and print data
timeIters = 100
part1list = msmrd2.integrators.particleList([part])
print('{:<10s}{:<10s}{:<40s}{:<40s}'.format("Iteration", "Time", "Position 1", "Orientation 1"))
for i in range(timeIters):
    print('{:<10d}{:<10f}{:<40s}{:<40s}'.format(i, integrator.clock, str(part1list[0].position), str(part1list[0].orientation)))
    integrator.integrate(part1list)